In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
# from scipy.integrate import solve_ivp

from defs import ring_calculation, add_lengths_to_df, Ring, CylinderSegment, integration

mpl.rcParams["lines.linewidth"] = 2.5
mpl.rcParams["font.size"] = 20
# mpl.rcParams('legend',fontsize=20) # using a size in points
plt.rc("legend", fontsize=17)  # using a size in points

In [ ]:
ngeom = 41

In [ ]:
### magnetic geometry

# to plot or not to plot (scans)
plt_on = False

# prepare a dataframe header
df = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)

# fixed R
R = 20  # radius in mm

# dR = 2  # width of the magnet in mm
# for dR in np.linspace(2, 10, 5):  # length in mm
for dR in [2]:  # length in mm
    for L in np.linspace(10, 90, ngeom):  # length in mm
        # for L in np.linspace(40, 60, 11):  # length in mm
        # for L in [10, 50, 90]:  # length in mm
        rlines, Brinterp, Bzinterp, parallel, zz, rr, zsep, ring = ring_calculation(
            R, L, dR, plt_on
        )  # calculate the field
        df = integration(
            R, L, dR, df, rlines, Brinterp, Bzinterp, parallel, zsep, plt_on
        )  # generate parameter df

In [ ]:
# Fig. 4


def format_func(value, tick_number):
    """pi as ticks"""
    # find number of multiples of pi/2
    N = int(np.round(2 * value / np.pi))
    if N == 0:
        return "0"
    elif N == 1:
        return r"$\pi/2$"
    elif N == 2:
        return r"$\pi$"
    elif N % 2 > 0:
        return rf"${N}\pi/2$"
    else:
        return rf"${N // 2}\pi$"


# analytical result
pb = df.R[0] * np.sqrt(6)  # parallelB value for R = 20mm


fig, ax = plt.subplots(figsize=(10, 7))

pp = [0.1, 0.3, 0.5, 0.9]

pas = np.array(df.parallelism.tolist(), dtype=float)
# num = pas.shape[0]
# for p in range(num - 1):
#    plt.plot(df.L, np.sin(pas[:, p]), label="p = " + str(pp[p]))  # *180/np.pi)

# plt.plot(df.L, np.tan(pas[:, 0]))
plt.plot(df.L, np.sin(pas[:]))
plt.axvline(x=pb, color="tab:olive")

# plt.yticks([-np.pi,0,np.pi])
plt.xlabel("L / mm")
# plt.ylabel(r"$B_r$ / $B_z$ ")
plt.ylabel(r"$\sin (\alpha)$ ")
# plt.legend()
ax.ticklabel_format(axis="y", scilimits=[-3, 3])

# ax.yaxis.set_major_locator(plt.MultipleLocator(np.pi))
# ax.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 2))
# ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))

fig.tight_layout()
plt.grid()
plt.savefig("parallelism.png", dpi=300)

In [ ]:
# Fig. 5

fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
ax2 = ax.twinx()

lns1 = ax.plot(df.L, df.zsep_L / df.L, label=r"$\widetilde{z}$")
ax.set_xlabel("L / mm")
# ax.set_ylabel("$(z_{sep}-L/2)/L$ / mm")
ax.set_ylabel(r"$\widetilde{z}$ / mm")
ax.grid()

# select only the uppermost field line from the df
nn = 20  # == nline+1  # TODO parameter from class
line = []
for rrr in range(ngeom):  # r range
    line.append(0 + (nn) * rrr)

# plt.plot(df.L[line], 1 - (df.va[line] / (R*R*L/2)))
lns2 = ax2.plot(df.L[line], df.va[line], "r", label=r"$\widetilde{V}$")  # /(R*R*0.5*L))
# ax2.set_ylabel("$\widetilde{V}$ = $V_{active}$ / $V_{total}$")
ax2.set_ylabel(r"$\widetilde{V}$")

lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs)
plt.tight_layout()

plt.savefig("zsep_va.png", dpi=300, bbox_inches="tight")

In [ ]:
df.to_csv("scan_l2r_mr.csv")

df = add_lengths_to_df(df)  # add lengths to df

# integral values for each thruster geometry
dfsum = df.groupby(["R", "L"]).sum().reset_index()
dfsum


In [ ]:
# Fig. 7


def plot_lengths(df, dfsum):
    """plot various length definitions"""
    pb = df.R[0] * np.sqrt(6)  # parallelB value for R = 20mm

    fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    nlines = 20
    rhos = [df.r0[2], df.r0[10], df.r0[17]]
    label = "average"

    # plot all lengths for all parameters
    ax[0].plot(dfsum.L, dfsum.length / nlines, label=label)
    ax[0].axvline(x=pb, color="tab:olive")
    ax[0].set_ylabel(r"$\ell$ / a.u.")
    ax[1].plot(dfsum.L, dfsum.mr / nlines, label=label)
    ax[1].axvline(x=pb, color="tab:olive")
    ax[1].set_ylabel(r"$R_m$")

    # exponentially weighted lengths
    ax[2].plot(dfsum.L, dfsum["r0_l_mr_exp_r0"] / nlines, label=label)
    ax[2].axvline(x=pb, color="tab:olive")
    ax[2].set_ylabel(r"$\tau$ / a.u.")
    # $\hat{\ell} = \ell \cdot \sqrt{R_m} \cdot \rho_0 \cdot e^{-\rho_0}$

    # for selected lines
    ls = ["-", ":", "--"]
    for i, l in enumerate(rhos):
        dff = df.where(np.abs(df.r0 - l) < 0.2).dropna()
        # plot all lengths for all parameters

        ax[0].plot(
            dff.L,
            dff.length,
            label=str(np.round(l, decimals=1)) + " mm",
            linestyle=ls[i],
        )
        ax[1].plot(
            dff.L, dff.mr, label=str(np.round(l, decimals=1)) + " mm", linestyle=ls[i]
        )
        ax[2].plot(
            dff.L,
            dff["r0_l_mr_exp_r0"],
            label=str(np.round(l, decimals=1)) + " mm",
            linestyle=ls[i],
        )

    # ax[0,0].legend()
    # ax[0,1].legend()
    # ax[1,0].legend()
    ax[1].legend()
    ax[2].set_xlabel("L / mm")


plot_lengths(df, dfsum)
plt.savefig("lengths.png", dpi=300)

In [ ]:
ndf = df.copy()  # normed df ->
ndf = add_lengths_to_df(ndf, ndf.L)

ndfsum = ndf.groupby(["R", "L"]).sum().reset_index()
ndfsum

plot_lengths(ndf, ndfsum)
plt.savefig("normed_lengths.png", dpi=300)

In [ ]:
### Fig. 3

# to plot or not to plot (scans)
plt_on = True

# prepare a dataframe header
df = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)


fig = plt.figure(constrained_layout=True, figsize=(11, 8), dpi=300)
fig.set_tight_layout(True)

gs = GridSpec(2, 6, figure=fig)
ax1 = fig.add_subplot(gs[0, :2])
ax2 = fig.add_subplot(gs[0, 2:])
ax3 = fig.add_subplot(gs[-1, :])

axes = [ax1, ax2, ax3]
label = ["(a)", "(b)", "(c)"]

# fixed R
R = 20  # radius in mm
for dR in [2]:  # length in mm
    for i, L in enumerate([10, 50, 90]):  # length in mm
        m = Ring(R, L, dR, plt_on, axes[i])
        df = integration(
            m.R, m.L, m.dR, df, m.r0, m.bri, m.bzi, m.pa, m.zsep, plt_on, axes[i]
        )  # generate parameter df
        axes[i].plot(m.zsep, 0, "k*", markersize=14)  # , color="tab:orange")
        axes[i].set_title(label[i], fontfamily="serif", loc="left", fontsize="medium")


plt.savefig("RL_field.png", dpi=300, bbox_inches="tight")

In [ ]:
### magnetic geometry
ngeom = 18

# to plot or not to plot (scans)
plt_on = False

# prepare a dataframe header
dfr = pd.DataFrame(
    columns=["R", "L", "dR", "parallelism", "zsep_L", "va", "r0", "mr", "length"]
)

# for all R,L
L = 50  # radius in mm

# for dR in np.linspace(2, 10, 5):  # width of the magnet in mm
for dR in [2]:  # length in mm
    for R in np.linspace(6, 40, ngeom):  # length in mm
        # for R in [6, 22, 40]:  # length in mm
        rlines, Brinterp, Bzinterp, parallel, zz, rr, zsep, ring = ring_calculation(
            R, L, dR, plt_on
        )  # calculate the field
        dfr = integration(
            R, L, dR, dfr, rlines, Brinterp, Bzinterp, parallel, zsep, plt_on
        )  # generate parameter df

In [ ]:
dfr

In [ ]:
dfr = add_lengths_to_df(dfr)

# dfr
dfrsum = dfr.groupby(["R", "L"]).sum().reset_index()
dfrsum

In [ ]:
# Fig. 8


def plot_lengths_R(df, dfsum):
    """plot various length definitions"""
    pb = df.L[0] / np.sqrt(6)  # parallelB value for L = 50mm

    fig, ax = plt.subplots(3, 1, figsize=(8, 8), sharex=True)

    nlines = 20
    # rhos = [df.r0[2], df.r0[10],df.r0[17]]  # fixed positions
    rhos = [2, 10, 17]  # relative positions
    rlabel = ["0.85", "0.5", "0.1"]

    label = "average"

    # plot all lengths for all parameters
    ax[0].plot(dfsum.R, dfsum.length / nlines, label=label)
    ax[0].axvline(x=pb, color="tab:olive")
    ax[0].set_ylabel(r"$\ell$ / a.u.")
    ax[1].plot(dfsum.R, dfsum.mr / nlines, label=label)
    ax[1].axvline(x=pb, color="tab:olive")
    ax[1].set_ylabel("$R_m$")

    # exponentially weighted lengths
    ax[2].plot(dfsum.R, dfsum["r0_l_mr_exp_r0"] / nlines, label=label)
    ax[2].axvline(x=pb, color="tab:olive")
    ax[2].set_ylabel(r"$\tau$ / a.u.")

    # for selected lines
    ls = ["-", ":", "--"]
    for i, l in enumerate(rhos):
        line = []
        for rrr in range(ngeom):  # number of R runs  TODO adapt!! 18
            line.append(l + (nlines) * rrr)
        dff = df.iloc[line]  # ???
        # plot all lengths for all parameters

        ax[0].plot(
            dff.R, dff.length, label=r"$r_0/R = $" + str(rlabel[i]), linestyle=ls[i]
        )
        ax[1].plot(dff.R, dff.mr, label=r"$r_0/R = $" + str(rlabel[i]), linestyle=ls[i])
        ax[2].plot(dff.R, dff["r0_l_mr_exp_r0"], label=str(rlabel[i]), linestyle=ls[i])

    ax[0].legend()
    ax[2].set_xlabel("R / mm")


plot_lengths_R(dfr, dfrsum)
plt.savefig("lengths_R_varied.png", dpi=300)

In [ ]:
# CylinderSegment is shorter, use instead!
from magpylib.magnet import Cylinder

# ring from CylinderSegment
ring0 = CylinderSegment(magnetization=(0, 0, 1000), dimension=(2, 4, 1, 0, 360))

# ring with cut-out
inner = Cylinder(magnetization=(0, 0, -1000), dimension=(4, 1))
outer = Cylinder(magnetization=(0, 0, 1000), dimension=(8, 1))
ring1 = inner + outer

print("getB from Cylindersegment", ring0.getB((1, 2, 4)))
print("getB from Cylinder cut-out", ring1.getB((1, 2, 4)))